#Chapter03 회귀 알로리즘과 모델 규제
농어의 무게를 예측하라!

###03-1 k-최근접 이웃 회귀

지도 학습 알고리즘
*   **분류** : 샘플을 몇 개의 클래스 중 하나로 분류하는 문제
*   **회귀** : 임의의 어떤 숫자를 예측하는 문제

**k-최근접 이웃 회귀** :

샘플에 가장 가까운 샘플 k개를 선택하여 이들의 (예를 들어) 평균을 구하는 알고리즘

데이터 준비

In [ ]:
import numpy as np

perch_length = np.array([8.4, 13.7, 15.0, 16.2, 17.4, 18.0, 18.7, 19.0, 19.6, 20.0, 21.0,   # 농어 길이
       21.0, 21.0, 21.3, 22.0, 22.0, 22.0, 22.0, 22.0, 22.5, 22.5, 22.7,
       23.0, 23.5, 24.0, 24.0, 24.6, 25.0, 25.6, 26.5, 27.3, 27.5, 27.5,
       27.5, 28.0, 28.7, 30.0, 32.8, 34.5, 35.0, 36.5, 36.0, 37.0, 37.0,
       39.0, 39.0, 39.0, 40.0, 40.0, 40.0, 40.0, 42.0, 43.0, 43.0, 43.5,
       44.0])
perch_weight = np.array([5.9, 32.0, 40.0, 51.5, 70.0, 100.0, 78.0, 80.0, 85.0, 85.0, 110.0, # 농어 무게
       115.0, 125.0, 130.0, 120.0, 120.0, 130.0, 135.0, 110.0, 130.0,
       150.0, 145.0, 150.0, 170.0, 225.0, 145.0, 188.0, 180.0, 197.0,
       218.0, 300.0, 260.0, 265.0, 250.0, 250.0, 300.0, 320.0, 514.0,
       556.0, 840.0, 685.0, 700.0, 700.0, 690.0, 900.0, 650.0, 820.0,
       850.0, 900.0, 1015.0, 820.0, 1100.0, 1000.0, 1100.0, 1000.0,
       1000.0])

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(perch_length, perch_weight)
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
# 훈련세트와 테스트세트로 나누기
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(perch_length, perch_weight, random_state=42)

In [ ]:
test_array = np.array([1,2,3,4])       # -> 1차원 배열
print(test_array.shape)

test_array = test_array.reshape(2,2)   # -> 2차원 배열로 바꾸기 reshape(): 넘파이 배열의 크기를 바꿔주는 메서드
print(test_array.shape)

In [ ]:
train_input = train_input.reshape(-1,1)  # reshape(-1,n): 열이 n인 크기로 바꿔줌, -1은 나머지 원소개수로 모두 채우라는 의미
test_input = test_input.reshape(-1,1)
print(train_input.shape, test_input.shape)

####**결정계수 (R²)**

대표적인 회귀 문제의 성능 측정 도구

score() 메서드는 분류의 경우, 테스트세트에 있는 샘플을 정확하게 분류한 개수의 비율이지만 

회귀에서는 결정계수로 계산이 된다.


In [ ]:
from sklearn.neighbors import KNeighborsRegressor  # KNeighborsClassifier과 매우 비슷

knr = KNeighborsRegressor()
knr.fit(train_input, train_target)  # 모델 훈련

print(knr.score(test_input, test_target))
# 회귀에서는 정확한 숫자를 맞힌다는 것은 거의 불가능 -> 타깃값이 임의의 수치이기 때문

결정계수 ( **R²** ) = 1 - ((타깃 - 예측)² 의 합) / ((타깃 - 평균)² 의 합)

1에 가까울수록 좋은 모델, 0에 가까울수록 그렇지 않다


In [ ]:
from sklearn.metrics import mean_absolute_error

test_prediction = knr.predict(test_input)   # 테스트세트에 대한 예측 생성

mae = mean_absolute_error(test_target, test_prediction)  # 테스트세트에 대한 '평균 절댓값 오차' 계산
print(mae)  # 평균적으로 오차가 19g 이다

####**과대적합 vs 과소적합**


*   *과대적합* : 테스트세트에서의 점수가 현격히 나쁜경우 --> 훈련세트에 '과대적합' 되었다
*   *과소적합* : 훈련세트보다 테스트세트의 점수가 높거나 모두 너무 낮은 경우 --> 모델이 너무 단순해서 훈련세트에 적절히 훈련되지 않았다



In [ ]:
print(knr.score(train_input, train_target))
# 훈련세트의 평가점수가 테스트세트 점수보다 낮다 --> '과소적합'

In [ ]:
# 해결방안: 모델을 좀 더 복잡하게 만든다 --> k-최근접 이웃 알고리즘에서 k의 개수를 조금만 줄인다
knr.n_neighbors = 3   # 이웃의 개수를 3으로 줄인다

knr.fit(train_input, train_target)
print(knr.score(train_input, train_target))   # 훈련세트 점수

In [ ]:
print(knr.score(test_input, test_target))   # 테스트세트 점수
# 점수가 비슷해졌다 -> '과소적합' 해결!

####**회귀 문제 다루기**

과소적합을 해결하려면 모델을 좀 더 복잡하게 --> k를 늘리기

과대적합을 해결하려면 모델을 좀 더 단순하게 --> k를 줄이기

In [ ]:
knr = KNeighborsRegressor()

x = np.arange(5,45).reshape(-1,1)

for n in [1,5,10]:      # k를 바꿔가면서 훈련
    knr.n_neighbors = n
    knr.fit(train_input, train_target)

    prediction = knr.predict(x)

    plt.scatter(train_input, train_target)
    plt.plot(x, prediction)
    plt.title(f'n_neighbors = {n}')
    plt.xlabel('length')
    plt.ylabel('weight')
    plt.show()

k가 커질수록 모델이 단순해지는 것을 알 수 있다

###03-2 선형 회귀

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

knr = KNeighborsRegressor(n_neighbors=3)

knr.fit(train_input, train_target)
print(knr.predict([[50]]))   # 실제 무게와 예측 무게가 차이가 난다

In [ ]:
import matplotlib.pyplot as plt

distances, indexes = knr.kneighbors([[50]])  # 50cm 농어의 이웃 구하기

plt.scatter(train_input, train_target)
plt.scatter(train_input[indexes], train_target[indexes], marker='D')  # 이웃 샘플만 다시 그림
plt.scatter(50, 1033, marker='^')   # 50cm 농어 데이터
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

삼각형의 이웃들의 평균으로 예측하는 것이므로 문제가 된다.

In [ ]:
print(np.mean(train_target[indexes]))

In [ ]:
print(knr.predict([[100]]))
# 새로운 샘플이 훈련세트의 범위를 벗어나면 엉뚱한 값으로 예측할 수도 있다

In [ ]:
import matplotlib.pyplot as plt

distances, indexes = knr.kneighbors([[100]])  # 50cm 농어의 이웃 구하기

plt.scatter(train_input, train_target)
plt.scatter(train_input[indexes], train_target[indexes], marker='D')  # 이웃 샘플만 다시 그림
plt.scatter(100, 1033, marker='^')   # 50cm 농어 데이터
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

####**선형 회귀**

널리 사용되는 대표적인 회귀 알고리즘 , 비교적 간단하고 성능이 뛰어남

In [ ]:
from sklearn.linear_model import LinearRegression   # 사이킷런의 선형 회귀 알고리즘

lr = LinearRegression()

lr.fit(train_input, train_target)

print(lr.predict([[50]]))

In [ ]:
print(lr.coef_, lr.intercept_)  # coef_: 기울기, intercept_: 절편 --> '모델 파라미터'



*   **모델 기반 학습** : 모델 파라미터를 찾기 위해 훈련하는 것
*   **사례 기반 학습** : 훈련 세트를 저장하는 것이 훈련의 전부인 것



In [ ]:
plt.scatter(train_input, train_target)

# plt.plot([x1,x2],[y1,y2]) -> (x1,y1)와 (x2,y2)를 방정식 그래프를 그림
plt.plot([15,50], [15*lr.coef_ + lr.intercept_, 50*lr.coef_ + lr.intercept_])

plt.scatter(50, 1241.8, marker='^')  # 50cm 농어 데이터
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
print(lr.score(train_input, train_target))   # 훈련세트
print(lr.score(test_input, test_target))     # 테스트세트

# 점수가 별로임  (모델이 단순함?)

####**다항 회귀**



*   단항식을 사용한 선형 회귀 : 최적의 직선을 찾는 것 --> ax + b
*   다항식을 사용한 선형 회귀 : 최적의 곡선을 찾는 것 --> ax² + bx + c

In [ ]:
# 농어의 길이를 제곱해서 원래 데이터 앞에 붙이기
train_poly = np.column_stack((train_input ** 2, train_input))
test_poly = np.column_stack((test_input ** 2, test_input))
# train_input ** 2 에도 넘파이 브로드캐스팅 적용

print(train_poly.shape, test_poly.shape)

In [ ]:
lr = LinearRegression()
lr.fit(train_poly, train_target)   # target값은 변형없이 그대로 사용

print(lr.predict([[50**2, 50]]))   # 농어 길이의 제곱과 원래 길이를 함께 넣어 주어야 함

In [ ]:
print(lr.coef_, lr.intercept_)

In [ ]:
point = np.arange(15,50)
plt.scatter(train_input, train_target)

plt.plot(point, 1.01*point**2 - 21.6*point + 116.05)   # 15에서 49까지의 2차 방정식 그래프를 그림

plt.scatter(50,1574,marker='^')  # 50cm 농어 데이터
plt.xlabel('length')
plt.ylabel('weight')
plt.show()

In [ ]:
print(lr.score(train_poly, train_target))
print(lr.score(test_poly, test_target))
# 점수가 크게 높아졌다

###03-3 특성 공학과 규제

####**다중 회귀**
 
여러 개의 특성을 사용한 선형 회귀


*   *특성 공학* : 기존의 특성을 사용해 새로운 특성을 뽑애내는 작업

예를 들어, '농어 길이 X 농어 높이' 라는 새로운 특성 생성



####데이터 준비

**판다스 (pandas)** : 

유명한 데이터 분석 라이브러리 , '*데이터프레임* ' 은 판다스의 핵심 데이터 구조이다
- CSV 파일 : 콤마 ( , ) 로 나누어진 텍스트 파일 , 판다스 데이터프레임을 만들기 위해 많이 사용하는 파일

 판다스의 ' read_csv( ) ' 함수에 주소를 넣어 주면 판다스에서 이 주소의 파일을 읽어준다

In [ ]:
import pandas as pd

df = pd.read_csv('https://bit.ly/perch_csv_data')  # 데이터프레임을 만든 후
perch_full = df.to_numpy()                         # 넘파이 배열로 바꿔준다
print(perch_full)

In [ ]:
from sklearn.model_selection import train_test_split

# 새로운 특성 perch_full로 훈련세트와 테스트세트 나누기
train_input, test_input, train_target, test_target = train_test_split(perch_full, perch_weight, random_state=42)

####사이킷런의 변환기

변환기 : 특성을 만들거나 전처리하기 위한 다양한 클래스 , 모두 fit( ), transform( ) 메서드를 제공

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures()
poly.fit([[2,3]])               # 2개의 특성 2와 3으로 이루어진 샘플 하나를 적용
print(poly.transform([[2,3]]))
# 절편은 값이 1인 특성과 곱해지는 계수로 보기 때문에 자동으로 1이 추가된다

In [ ]:
poly = PolynomialFeatures(include_bias=False)   # include_bias=False: 절편을 위한 항을 제거
poly.fit([[2,3]])
print(poly.transform([[2,3]]))

In [ ]:
# 위 방식을 train_input에 적용
poly = PolynomialFeatures(include_bias=False)
poly.fit(train_input)
train_poly = poly.transform(train_input)
print(train_poly.shape)

In [ ]:
poly.get_feature_names()  # 9개의 특성이 각각 어떤 입력의 조합으로 만들어졌는지 알려줌

In [ ]:
test_poly = poly.transform(test_input)   # 훈련세트로 학습한 변환기로 테스트세트까지 변환해주어야 함!

####다중 회귀 모델 훈련하기

In [ ]:
from sklearn.linear_model import LinearRegression

lr = LinearRegression()
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))
# 특성이 늘어나면 선형 회귀의 능력은 매우 강해진다!

In [ ]:
print(lr.score(test_poly, test_target))

In [ ]:
poly = PolynomialFeatures(degree=5, include_bias=False)   # degree=n : 최고차항의 최대차수를 n으로 지정
poly.fit(train_input)
train_poly = poly.transform(train_input)
test_poly = poly.transform(test_input)
print(train_poly.shape)

In [ ]:
lr.fit(train_poly, train_target)
print(lr.score(train_poly, train_target))

In [ ]:
print(lr.score(test_poly, test_target))
# 특성의 개수를 늘려서 모델을 강력하게 만들었기 때문에 훈련세트에 너무 과대적합되었다!

####규제
모델이 훈련세트에 과대적합되지 않도록 만드는 것 --> 선형 회귀 모델의 경우, 계수의 크기를 줄이는 일이다

In [ ]:
# 선형회귀모델에 규제를 할 때 계수 값의 크기를 맞추어 주기 위해 정규화를 해야 함
from sklearn.preprocessing import StandardScaler  # 정규화 해주는 클래스

ss = StandardScaler()   
ss.fit(train_poly)
train_scaled = ss.transform(train_poly)
test_scaled = ss.transform(test_poly)

선형 회귀 모델에 규제를 추가한 모델 :

*   **릿지 회귀** : 계수를 제곱한 값을 기준으로 규제를 적용함
*   **라쏘 회귀** : 계수의 절댓값을 기준으로 규제를 적용함

 사이킷런에서 이 두 알고리즘을 모두 제공한다

일반적으로 릿지를 조금 더 선호한다.  라쏘는 계수를 크기를 줄일 때 0으로 만들 수 있다


####**릿지 회귀**



In [ ]:
from sklearn.linear_model import Ridge
ridge = Ridge()
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))

In [ ]:
print(ridge.score(test_scaled, test_target))

릿지, 라쏘 모델을 사용할 때 규제의 양을 임의로 조절 가능 --> alpha 매개변수로

**하이퍼파라미터** : 머신러닝 모델이 학습할 수 없고 사람이 알려줘야 하는 파라미터

In [ ]:
# 릿지모델의 적절한 alpha값을 찾기 --> 훈련세트와 테스트세트의 점수가 가장 가까운 지점이 최적의 alpha값이다
import matplotlib.pyplot as plt

train_score = []
test_score = []
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]

for alpha in alpha_list:
    ridge = Ridge(alpha=alpha)
    ridge.fit(train_scaled, train_target)
    train_score.append(ridge.score(train_scaled, train_target))  # 훈련세트 점수 저장
    test_score.append(ridge.score(test_scaled, test_target))     # 테스트세트 점수 저장

In [ ]:
plt.plot(np.log10(alpha_list), train_score)  # 보기 좋게 스케일링
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()      # 파란색이 훈련세트, 주황색이 테스트세트

왼쪽으로 갈수록 과대적합, 오른쪽으로 갈수록 과소적합

최적의 alpha값 : 0.1 임을 알 수 있다

In [ ]:
ridge = Ridge(alpha=0.1)    # 최적의 하이퍼파라미터값
ridge.fit(train_scaled, train_target)
print(ridge.score(train_scaled, train_target))
print(ridge.score(test_scaled, test_target))

####**라쏘 회귀**
릿지와 매우 비슷


In [ ]:
from sklearn.linear_model import Lasso

lasso = Lasso()
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))

In [ ]:
print(lasso.score(test_scaled, test_target))

In [ ]:
train_score = []
test_score = []
alpha_list = [0.001, 0.01, 0.1, 1, 10, 100]

for alpha in alpha_list:
    lasso = Lasso(alpha=alpha, max_iter=10000)    # 최적의 계수를 찾기 위해 반복적인 계산을 수행, max_iter= : 최대 반복 횟수
    lasso.fit(train_scaled, train_target)
    train_score.append(lasso.score(train_scaled, train_target))
    test_score.append(lasso.score(test_scaled, test_target))

In [ ]:
plt.plot(np.log10(alpha_list), train_score)
plt.plot(np.log10(alpha_list), test_score)
plt.xlabel('alpha')
plt.ylabel('R^2')
plt.show()      # 파란색이 훈련세트, 주황색이 테스트세트

왼쪽으로 갈수록 과대적합, 오른쪽으로 갈수록 과소적합

최적의 alpha값 : 10 임을 알 수 있다

In [ ]:
lasso = Lasso(alpha=10)
lasso.fit(train_scaled, train_target)
print(lasso.score(train_scaled, train_target))
print(lasso.score(test_scaled, test_target))

In [ ]:
# 라쏘 회귀는 릿지와 달리 계수 값을 아예 0으로 만들 수도 있다

print(np.sum(lasso.coef_ == 0))   # 라쏘 모델의 계수가 0인 것을 헤아려줌